In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import time

/Users/josoga2/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/tt/tj2184d15757rp2nc210b8tm0000gn/T/ipykernel_8503/1423607470.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
title = 'Bioinformatics'
location = 'Toronto'
#special string = 'Toronto%2C%2BOntario%2C%2BCanada'

In [3]:
#list
list_URL = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&start=15'

response = requests.get(list_URL)
list_data = response.text
list_soup = BeautifulSoup(list_data, 'html.parser')
page_jobs = list_soup.find_all('li')



In [4]:
id_list = []

for job in page_jobs:
    base_card_div = job.find('div', {'class': 'base-card'})
    job_id = base_card_div.get('data-entity-urn').split(':')[3]
    #print(job_id)
    id_list.append(job_id)
    

In [5]:
#job
job_list = []

for job_id in id_list:
    job_url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}'
    job_response = requests.get(job_url)
    print(job_response.status_code)
    job_soup = BeautifulSoup(job_response.text, 'html.parser')
    job_post = {}
    
    try:
        job_post['Job_title'] = job_soup.find('h2', {'class': 'top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title'}).text.strip()
    except:
        job_post['Job_title'] = None
    try:
        job_post['company_name'] = job_soup.find('a', {'class': "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post['company_name'] = None
        
    try:
        job_post['time_posted'] = job_soup.find('span', {'class': "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post['time_posted']= None
    try:
        job_post['num_applicants'] = job_soup.find('span', {'class': "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
    except:
        job_post['num_applicants'] = None
        
    job_list.append(job_post)



200
200
200
200
200
200
200
429
200
200


## function

In [6]:
def linkMaker(title = '', location = '', lims = 100):
    rangeLims = range(1 ,lims+1 , 10)
    
    all_URL = []
    for istop in rangeLims:
        for ititle in title:
            for iloc in location:
                list_URL = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={ititle}&location={iloc}&start={istop}"
                all_URL.append(list_URL)
    
    return all_URL
        
        
len(linkMaker(title= ['Bioinformatics', 'Biomarker'], location = ['California', 'Georgia']))


40

In [7]:
def scrapy(url_Main):
    
    
    #get responses
    response = requests.get(url_Main)
    list_data = response.text
    list_soup = BeautifulSoup(list_data, 'html.parser')
    page_jobs = list_soup.find_all('li')
    
    #gather ID_LIST
    id_list = []

    for job in page_jobs:
        base_card_div = job.find('div', {'class': 'base-card'})
        #print(base_card_div)
        if base_card_div == None:
            job_id = None
            id_list.append(job_id)
        else:
            job_id = base_card_div.get('data-entity-urn').split(':')[3]
            #print(job_id)
            id_list.append(job_id)
        
    #get the job details
    #job
    job_list = []

    for job_id in id_list:
        job_url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}'
        job_response = requests.get(job_url)
        #print(job_response.status_code)
        job_soup = BeautifulSoup(job_response.text, 'html.parser')
        job_post = {}

        try:
            job_post['Job_title'] = job_soup.find('h2', {'class': 'top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title'}).text.strip()
        except:
            job_post['Job_title'] = None
        try:
            job_post['company_name'] = job_soup.find('a', {'class': "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
        except:
            job_post['company_name'] = None
            
        try:
            job_post['location'] = job_soup.find('span', {'class': "topcard__flavor topcard__flavor--bullet"}).text.strip()
        except:
            job_post['location']= None

        try:
            job_post['time_posted'] = job_soup.find('span', {'class': "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            job_post['time_posted']= None
        try:
            job_post['num_applicants'] = job_soup.find('span', {'class': "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip().split(" ")[0]
        except:
            job_post['num_applicants'] = None
        
        try:
            job_post['seniority'] = job_soup.find('ul', {'class': "description__job-criteria-list"}).text.strip().split('\n')[3]
            job_post['employment'] = job_soup.find('ul', {'class': "description__job-criteria-list"}).text.strip().split('\n')[11]
            job_post['function'] = job_soup.find('ul', {'class': "description__job-criteria-list"}).text.strip().split('\n')[19]
            job_post['industries'] = job_soup.find('ul', {'class': "description__job-criteria-list"}).text.strip().split('\n')[27]
        except:
            job_post['seniority'] = None
            job_post['employment'] = None
            job_post['function'] = None
            job_post['industries'] = None
            #salary compensation__salary
        try:
            job_post['salary'] = job_soup.find('div', {'class': "compensation__salary"}).text.strip().split("-")[0]
        except:
            job_post['salary'] = None
        
        try:
            job_post['description'] = job_soup.find('section', {'class': "show-more-less-html"}).text.strip()
        except:
            job_post['description'] = None
            

        job_list.append(job_post)
        
        time.sleep(5)
    
    print(f'processing {len(job_list)} job list')
    return pd.DataFrame(job_list)


#testing_sc = scrapy(linkMaker(title= ['Bioinformatics'], location = ['California'])[0])    
testing_sc = scrapy('https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Berlin&start=1')

processing 10 job list


In [8]:
testing_sc

,Job_title,company_name,location,time_posted,num_applicants,seniority,employment,function,industries,salary,description
0,Assay Technology Pathologist (All Genders),Bayer,"Berlin, Berlin, Germany",1 week ago,100,Entry level,Full-time,Health Care Provider,"Chemical Manufacturing, Pharmaceut...",None,"At Bayer we’re visionaries, driven to solve th..."
1,"Senior Principal Scientist, Oncology Research ...",Bayer,"Berlin, Berlin, Germany",1 week ago,48,Mid-Senior level,Full-time,"Research, Analyst, and Informati...","Chemical Manufacturing, Pharmaceut...",None,"At Bayer we’re visionaries, driven to solve th..."
2,Translation Imaging Lead (all genders),Bayer,"Berlin, Berlin, Germany",6 days ago,None,Mid-Senior level,Full-time,"Marketing, Public Relations, and...","Chemical Manufacturing, Pharmaceut...",None,"At Bayer we’re visionaries, driven to solve th..."
3,Specialist* Custom Peptide Synthesis,JPT Peptide Technologies,"Berlin, Berlin, Germany",5 days ago,29,Entry level,Full-time,Management and Manufacturing,Biotechnology Research,None,Werde Teil unseres Teams voller Pioniere! Als ...
4,Medical Affairs Specialist (m/w/d),myGwork - LGBTQ+ Business Community,"Hennigsdorf, Brandenburg, Germany",1 week ago,None,Entry level,Full-time,"Research, Analyst, and Informati...",Biotechnology,None,This inclusive employer is a member of myGwork...
5,None,None,None,None,None,None,None,None,None,None,None
6,Medical Affairs Specialist (m/w/d),Thermo Fisher Scientific,"Hennigsdorf, Brandenburg, Germany",4 days ago,46,Not Applicable,Full-time,"Research, Science, and Engineering",Pharmaceutical Manufacturing and B...,None,Thermo Fisher Scientific Inc. is the world lea...
7,(Junior) Projektmanager:in Schwerpunkt Ringver...,Quality in Pathology,"Berlin, Germany",2 weeks ago,None,Entry level,Full-time,Project Management and Informati...,Public Health,None,Die Qualitätssicherungs-Initiative Pathologie ...
8,Statistician/Biostatistician Clinical Research...,myGwork - LGBTQ+ Business Community,"Hennigsdorf, Brandenburg, Germany",1 week ago,None,Mid-Senior level,Full-time,"Research, Analyst, and Informati...",Biotechnology,None,This inclusive employer is a member of myGwork...
9,Founder's Associate,aescolab,"Berlin, Germany",3 weeks ago,176,None,None,None,None,None,WHAT WE DOaescolab provides high-quality blood...


In [9]:
def dfMAKER(lmOut):
    
    """ 
        lmout is the output of linkmaker
    """
    
    #lmOut_ind = list(range(1, len(lmOut)+1))
    
    dfOut = scrapy(lmOut[0])
    
    for ind in list(range(1, len(lmOut))):
        print(lmOut[ind])
        dfOut = pd.concat([dfOut, scrapy(lmOut[ind])], ignore_index=True)
        
    return(dfOut)

#dfMAKER(linkMaker(title= 'Bioinformatics', location = 'California', lims=20))

### Testing

In [10]:
#create links
creatingLinks = linkMaker(title= ['Bioinformatics', 
                                  'Biomarker', 
                                  'Drug%2BDiscovery', 
                                  'NGS'], 
                          
                          location = ['California', 
                                      'Georgia', 
                                      'Massachusetts', 
                                      'New%2BYork',
                                      'Chicago',
                                      'New%2BJersey',
                                      'Washington', 
                                      'Frankfurt', 
                                      'Berlin', 
                                      'Munich', 
                                      'Darmstadt', 
                                      'London', 
                                      'Cambridge',
                                      'Oxford', 
                                      'Lagos', 
                                      'Johannesburg',
                                      'Accra', 
                                      'Amsterdam', 
                                      'Paris', 
                                      'Manchester', 
                                      'Tokyo',
                                      'Beijing'])
len(creatingLinks)

880

In [11]:
random.sample(creatingLinks, k=1)
#creatingLinks

['https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=London&start=11']

In [12]:
# become a mess
bfx_jobs_data = dfMAKER(creatingLinks)

processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Georgia&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Massachusetts&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=New%2BYork&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Chicago&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=New%2BJersey&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Washington&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&locatio

processing 3 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Johannesburg&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Accra&start=1
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Amsterdam&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Paris&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Manchester&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Tokyo&start=1
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location

processing 2 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Berlin&start=11
processing 8 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Munich&start=11
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Darmstadt&start=11
processing 2 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=London&start=11
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Cambridge&start=11
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Oxford&start=11
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Lagos&start=11
processing 0 job list
https://www.

processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Georgia&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Massachusetts&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=New%2BYork&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Chicago&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=New%2BJersey&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Washington&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&l

processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Johannesburg&start=21
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Accra&start=21
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Amsterdam&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Paris&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Manchester&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Tokyo&start=21
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&loc

...

processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Johannesburg&start=61
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Accra&start=61
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Amsterdam&start=61
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Paris&start=61
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Manchester&start=61
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Tokyo&start=61
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&locat

processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Berlin&start=71
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Munich&start=71
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Darmstadt&start=71
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=London&start=71
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Cambridge&start=71
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Oxford&start=71
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Lagos&start=71
processing 0 job list
https://www.li

processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Georgia&start=81
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Massachusetts&start=81
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=New%2BYork&start=81
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Chicago&start=81
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=New%2BJersey&start=81
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&location=Washington&start=81
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Bioinformatics&lo

processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Johannesburg&start=81
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Accra&start=81
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Amsterdam&start=81
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Paris&start=81
processing 4 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Manchester&start=81
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&location=Tokyo&start=81
processing 10 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Drug%2BDiscovery&locati

processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Berlin&start=91
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Munich&start=91
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Darmstadt&start=91
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=London&start=91
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Cambridge&start=91
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Oxford&start=91
processing 0 job list
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Biomarker&location=Lagos&start=91
processing 0 job list
https://www.li

In [13]:
bfx_jobs_data

,Job_title,company_name,location,time_posted,num_applicants,seniority,employment,function,industries,salary,description
0,Research Scientist - Bioinformatics III*,Cube Hub Inc.,"California, United States",3 months ago,45,Mid-Senior level,Contract,Other,Staffing and Recruiting,None,DescriptionResearch Scientist (Bioinformatics ...
1,Bioinformatics Scientist,"Centrillion Technology, Inc.","Palo Alto, CA",2 months ago,187,Mid-Senior level,Full-time,"Research, Analyst, and Informati...",Biotechnology Research,"$75,000.00/yr",Centrillion is at the forefront of genomics an...
2,Single Cell Computational Biologist,Vevo Therapeutics,"South San Francisco, CA",1 month ago,153,Not Applicable,Full-time,"Research, Analyst, and Informati...",Biotechnology Research,"$120,000.00/yr",About Vevo TherapeuticsWe are a biotechnology ...
3,Scientist I/II - R&D,"Eitr Biologics, Inc.","San Diego, CA",1 week ago,96,Mid-Senior level,Full-time,Research,Non-profit Organizations and Prima...,"$100,000.00/yr","Eitr Biologics, Inc., (Eitr) is a San Diego, C..."
4,"Principal Scientist, Computational Biology",Genesis Therapeutics,"San Diego, CA",2 months ago,None,Not Applicable,Full-time,"Research, Analyst, and Informati...","Transportation, Logistics, Supply ...",None,Genesis Therapeutics is pioneering artificial ...
...,...,...,...,...,...,...,...,...,...,...,...
3020,"Sr. Sales Director, Biopharma",BostonGene,"Boston, MA",2 days ago,191,Executive,Full-time,"Sales, Science, and Business Dev...","Pharmaceutical Manufacturing, Biot...",None,About BOSTONGENEBostonGene Corporation is pion...
3021,"Principal Scientist, Clinical Genetics and Gen...",Novartis,"Cambridge, MA",1 day ago,None,Mid-Senior level,Full-time,"Research, Analyst, and Informati...",Pharmaceutical Manufacturing,None,SummaryAbout the role: In this laboratory-focu...
3022,"Director Research, Antibody Discovery and Opti...",Takeda,"Boston, MA",2 days ago,None,Mid-Senior level,Full-time,"Research, Analyst, and Informati...",Pharmaceutical Manufacturing,"$149,100.00/yr","By clicking the “Apply” button, I understand t..."
3023,Director of Field Applications Sciences,Opentrons Labworks Inc.,"Boston, MA",5 days ago,None,Director,Full-time,"Research, Analyst, and Informati...","Biotechnology Research, Research S...","$200,000.00/yr","Job Title: Director, Field Applications Scienc..."


In [15]:
bfx_jobs_data.to_csv("bfx_jobs_data_30_06_24.csv", index=False)